In [1]:
import os 
import pandas as pd
import re
import numpy as np

In [2]:
name_pattern = '(?s:.*)(ssl|init|lfs|delay)'
probability_pattern  = '\d\.\d{1,2}'

In [3]:
def get_type(x):
    try:
        return re.search(name_pattern,x).group(1)
    except Exception as e:
        return 'init'

def get_probability(x):
    proba = re.findall(probability_pattern,x)
    if proba:
        return proba[-1]
    

In [4]:

def prepare_logs(logs_file,info_from_hyperparameters = []):

    # read data
    res = pd.read_csv(os.path.join('logs',logs_file),header= None)

    # renaming columns
    res = res.rename(columns={0:'stream',1:'hyperparameters',2:'B',3:'frequency'})
    res['hyperparameters'] = res['hyperparameters'].apply(lambda x: dict([i.split(";") for i in re.findall( '\((.*?)\)',x)]))
    B = res['B'].iloc[0]
    cl = res.iloc[:,B+6:].columns
    res = res.rename(columns = dict(zip(res.columns[4:B+6],[f"B_{i}" for i in range(1,B+3)])))
    res = res.rename(columns = dict(zip(cl,[f"m_{i}" for i in range(len(cl))])))

    # adding new columns
    res['date'] = res['stream'].apply(lambda x: x.split(' - ')[0])
    res['stream'] = res['stream'].apply(lambda x: x.split(' - ')[1])
    res['type'] = res['stream'].apply(lambda x :  get_type(x))
    res['probability'] = res['stream'].apply(lambda x :  get_probability(x))
    
    for inf in info_from_hyperparameters:
        res['classifier'] = res['hyperparameters'].apply(lambda x : 'CluStream' if 'classifier' not in list(x.keys()) else 'ICLC' if 'classifier_n_models' in list(x.keys()) else 'ICLC_HT'  if 'classifier_grace_period' in list(x.keys()) 
                                                         else 'ICLC_NB' if 'nu' in list(x.keys())  else x['classifier'])
        res['threshold'] = res['hyperparameters'].apply(lambda x : None if 'threshold' not in list(x.keys()) else x['threshold'])
    # res['classifier'] = 'CluStream'
    # res['threshold'] = 0.1



    #change order
    new_order = list(res.columns[:2])
    new_order.extend(res.columns[-(3+len(info_from_hyperparameters)):])
    new_order.extend(res.columns[2:-(3+len(info_from_hyperparameters))])
    res = res[new_order]

    return res


In [5]:
def save_formatted_logs(res,file):
    res.to_csv(os.path.join("logs_formatted",file),index = False)

In [6]:
def concatenate_res(res,file):
    new_res = pd.read_csv(os.path.join("logs_formatted",file))
    res = pd.concat([new_res,res])
    res.to_csv(os.path.join("logs_formatted",file),index = False)

In [7]:
mapper = dict(zip(["<class 'river.forest.adaptive_random_forest.ARFClassifier'>",
 "<class 'river.tree.hoeffding_adaptive_tree_classifier.HoeffdingAdaptiveTreeClassifier'>",
 "<class 'river.naive_bayes.gaussian.GaussianNB'>", "<class 'river.dummy.PriorClassifier'>","<class 'river.dummy.NoChangeClassifier'>","CluStream"],['ARF','HT','NB','Majority','NoChange',"CluStream"]))

In [8]:
os.listdir('logs')

['01_08_2023_22_14_55_Accuracy_LED_gradual.log',
 '01_08_2023_22_19_21_Accuracy_LED_abrupt.log',
 '01_08_2023_22_38_13_Accuracy_RBF_fast.log',
 '02_08_2023_17_49_57_Accuracy_LED_abrupt.log',
 '02_08_2023_19_51_27.log',
 '02_08_2023_19_51_49.log',
 '31_07_2023_17_54_25_Accuracy_LED_abrupt.log',
 '31_07_2023_17_55_01_Accuracy_LED_abrupt.log',
 '31_07_2023_18_05_52_Accuracy_RBF_moderate.log',
 '31_07_2023_18_06_12_Accuracy_RBF_moderate.log',
 '31_07_2023_18_06_31_Accuracy_RBF_moderate.log',
 '31_07_2023_22_13_25_Accuracy_LED_gradual.log',
 '31_07_2023_22_15_36_Accuracy_RBF_fast.log',
 '31_07_2023_22_16_26_Accuracy_RBF_fast.log']

In [40]:

logs_file = os.listdir('logs')[14]
print(logs_file)
res = prepare_logs(logs_file,['classifier','threshold'])


res

IndexError: list index out of range

In [39]:
# groups = res.groupby('stream_short')
stream_name = 'RBF_fast'
metric = ''
delay_tyep = '_random'
# for gr1, gr_idx1 in groups:
#     print(gr1)
res = res.iloc[:,:-200]
#res = res[res['type']=='ssl']
groups2 = res.groupby('classifier')
for gr, gr_idx in groups2:
    classifier_name = mapper.get(gr,gr)
    print(classifier_name)
    print(gr_idx['stream'])
    # stream_short = gr_idx['stream_short'].iloc[0]
    # gr_idx = gr_idx.drop(columns=['stream_short'])
    try:
        concatenate_res(gr_idx,f'{stream_name}\\{classifier_name}{metric}{delay_tyep}.csv')
    except Exception:
        save_formatted_logs(gr_idx,f'{stream_name}\\{classifier_name}{metric}{delay_tyep}.csv')

ARF
1    RBF_fast_0_20000_random_delay
Name: stream, dtype: object
NB
2    RBF_fast_0_20000_random_delay
Name: stream, dtype: object
HT
0    RBF_fast_0_20000_random_delay
Name: stream, dtype: object


In [241]:
concatenate_res(res.drop(columns='threshold').iloc[:4,:-200],"LED_gradual\\CluStream.csv")

In [163]:
save_formatted_logs(res.iloc[6:,:-200],'LED_abrupt\\CluStream.csv')

In [ ]:
res.iloc[28]['hyperparameters']['classifier']

"<class 'river.naive_bayes.gaussian.GaussianNB'>"

In [116]:
old_res = pd.read_csv(os.path.join("logs_formatted",'CoverType\\CluStream.csv'))

In [117]:
old_res = old_res[old_res['threshold']!=1]

In [115]:
res.iloc[8:16,:-200]

,stream,hyperparameters,date,type,probability,classifier,threshold,B,frequency,B_1,...,m_190,m_191,m_192,m_193,m_194,m_195,m_196,m_197,m_198,m_199
8,LED_gradual_delay_0_20000,"{'threshold': '1', 'train_period': '0', 'class...",26-May-23 01:28:34,delay,None,<class 'river.naive_bayes.gaussian.GaussianNB'>,1,50,100,0.650840,...,0.73,0.69,0.75,0.77,0.72,0.77,0.77,0.7,0.71,0.78
9,initail_LED_Drift_gradual,"{'threshold': '1', 'train_period': '0', 'class...",25-May-23 23:49:41,init,None,<class 'river.naive_bayes.gaussian.GaussianNB'>,1,50,100,0.000000,...,0.73,0.69,0.75,0.77,0.72,0.77,0.77,0.7,0.71,0.78
10,LED_gradual_constant_delay_lfs_0.1_0_20000,"{'threshold': '1', 'train_period': '0', 'class...",25-May-23 23:49:35,lfs,0.1,<class 'river.naive_bayes.gaussian.GaussianNB'>,1,50,100,0.000000,...,,,,,,,,,,
11,LED_gradual_constant_delay_lfs_0.2_0_20000,"{'threshold': '1', 'train_period': '0', 'class...",26-May-23 00:01:39,lfs,0.2,<class 'river.naive_bayes.gaussian.GaussianNB'>,1,50,100,0.000000,...,,,,,,,,,,
12,LED_gradual_constant_delay_lfs_0.5_0_20000,"{'threshold': '1', 'train_period': '0', 'class...",26-May-23 00:08:43,lfs,0.5,<class 'river.naive_bayes.gaussian.GaussianNB'>,1,50,100,0.000000,...,,,,,,,,,,
13,LED_gradual_constant_delay_ssl_0.1_0_20000,"{'threshold': '1', 'train_period': '0', 'class...",26-May-23 05:58:19,ssl,0.1,<class 'river.naive_bayes.gaussian.GaussianNB'>,1,50,100,0.650095,...,,,,,,,,,,
14,LED_gradual_constant_delay_ssl_0.2_0_20000,"{'threshold': '1', 'train_period': '0', 'class...",26-May-23 08:00:28,ssl,0.2,<class 'river.naive_bayes.gaussian.GaussianNB'>,1,50,100,0.647374,...,,,,,,,,,,
15,LED_gradual_constant_delay_ssl_0.5_0_20000,"{'threshold': '1', 'train_period': '0', 'class...",26-May-23 10:43:42,ssl,0.5,<class 'river.naive_bayes.gaussian.GaussianNB'>,1,50,100,0.639145,...,,,,,,,,,,
